In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [94]:
data_path = "/kaggle/input/nyc-taxi-trip-duration"
train_df = pd.read_csv(
    os.path.join(data_path, "train.zip"),
    usecols=[
        "pickup_datetime",
        "pickup_longitude",
        "pickup_latitude",
        "dropoff_longitude",
        "dropoff_latitude",
        "trip_duration",
    ],
)
train_df.head(5)

Data Cleaning (WIP)

In [95]:
train_df = train_df[(train_df["trip_duration"] > 60) & (train_df["trip_duration"] < 10800)]

Create Features

In [96]:
def transform_dt(df):
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], format="%Y-%m-%d %H:%M:%S")
    df["pickup_month"] = df["pickup_datetime"].dt.month
    df["pickup_day"] = df["pickup_datetime"].dt.day
    df["pickup_weekday"] = df["pickup_datetime"].dt.weekday
    df["pickup_hour"] = df["pickup_datetime"].dt.hour
    df["pickup_minute"] = df["pickup_datetime"].dt.minute
    return df
train_df = transform_dt(train_df)
train_df["trip_duration"] = train_df["trip_duration"].apply(lambda x: round(x / 60))
train_df.head(5)

In [97]:
# Split into training and evaluation sets
def feature_selection(df):
    return df[[
        "pickup_longitude",
        "pickup_latitude",
        "dropoff_longitude",
        "dropoff_latitude", 
        "pickup_month",
        "pickup_day",
        "pickup_weekday",
        "pickup_hour",
        "pickup_minute"
    ]]
X = feature_selection(train_df).values
y = train_df["trip_duration"].values
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.20, random_state=1)

Linear Regression

In [98]:
models = []
models.append(("Linear Regression", LinearRegression()))
models.append(("XGBoost Regression", XGBRegressor()))
models.append(("LightGBM Regression", LGBMRegressor()))
models.append(("Support Vector Regression", LinearSVR()))
results = []
model_names = [model[0] for model in models]

Cross validation (very slow)

In [99]:
# for model_name, model in models:
#     kfold = KFold(n_splits=10, random_state=1, shuffle=True)
#     cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring="neg_mean_absolute_error", n_jobs=-1)
#     results.append(cv_results)
#     print(model_name, cv_results.mean(), cv_results.std())

Model Evaluation

In [100]:
# for model_name, model in models:
#     model.fit(X_train, Y_train)
#     Y_pred = model.predict(X_val)
#     print(f"{model_name}: {mean_absolute_error(Y_pred, Y_val)}")

In [101]:
# Choosing LightGBM 
model = LGBMRegressor()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
print("Mean Absolute Error:", mean_absolute_error(Y_pred, Y_val))

# Output serialized model
model_file = "/kaggle/working/final_model.pickle"
pickle.dump(model, open(model_file, "wb"))

In [107]:
# Import test data
test_df = pd.read_csv(
    os.path.join(data_path, "test.zip"),
    usecols=[
        "pickup_datetime",
        "pickup_longitude",
        "pickup_latitude",
        "dropoff_longitude",
        "dropoff_latitude",
        # "trip_duration",
    ],
)
test_df = transform_dt(test_df)
X_test = feature_selection(test_df).values

# Import serialized model and compute
loaded_model = pickle.load(open(model_file, "rb"))
Y_test = loaded_model.predict(X_test)